## Import Packages

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal, kl_divergence
import pytorch_lightning as pl
import tensorboard
import pandas as pd
import scanpy as sc
import scvi
from scvi.nn import Encoder
from scvi.module._peakvae import Decoder
from scvi.dataloaders import DataSplitter
from scvi.dataloaders._ann_dataloader import AnnDataLoader
import sklearn

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
user = "Sabine"

## Load the data

In [ ]:
if user == "Tobi":
    data_path = 'C:/Users/Tobias/Desktop/Single Cell Data/Full/phase2-private-data/common/openproblems_bmmc_multiome_phase2'
if user == "Sabine":
    data_path = "/mnt/data/output/datasets/common/openproblems_bmmc_multiome_phase2"
    
only_train = True

adata = sc.read_h5ad(os.path.join(data_path, "openproblems_bmmc_multiome_phase2.manual_formatting.output_mod2.h5ad"))
gex = sc.read_h5ad(os.path.join(data_path, "openproblems_bmmc_multiome_phase2.manual_formatting.output_rna.h5ad"))

if only_train == True:
    test_adata = adata[adata.obs["is_train"] == False]
    test_gex = gex[gex.obs["is_train"] == False]
    
    adata = adata[adata.obs["is_train"] == True]
    gex = gex[gex.obs["is_train"] == True]

In [ ]:
adata.obsm['X_gex'] = gex.layers['counts']
test_adata.obsm['X_gex'] = test_gex.layers['counts']

In [ ]:
import h5py
if user == "Tobi":
   binding_probs = pd.read_hdf('C:/Users/Tobias/Desktop/Single Cell Data/deepbind_output.h5ad', key = 'ChIP-seq/atac-peaks') 
if user == "Sabine":
   binding_probs = pd.read_hdf("/mnt/data/output/deepbind/deepbind_output.h5ad", key = 'ChIP-seq/atac-peaks')

TF_names = binding_probs.columns


In [ ]:
binding_probs = torch.tensor(binding_probs.values)
binding_probs = torch.sigmoid(binding_probs) #change logits to binding_probs
binding_probs.shape


## Set up the model

In [ ]:
class GexToATAC(pl.LightningModule):

    def __init__(self, hparams):
        super().__init__()  
        
        self.encoder = Encoder(
            n_input=13431,
            n_layers=hparams["n_layers"],
            n_output=hparams["latent_dim"],
            n_hidden=hparams["n_hidden_encoder"],
            n_cat_list=None,
            dropout_rate=0.1,
            activation_fn=torch.nn.LeakyReLU,
            distribution="normal",
            var_eps=0,
            use_batch_norm=False,
            use_layer_norm=True,
        )
        
        self.bp = nn.Parameter(hparams["binding_probabilities"], requires_grad=False)
        self.bias = nn.Parameter(torch.rand(1, self.bp.shape[0]))
        self.Sigmoid = nn.Sigmoid()
        
    def forward(self, x, use_z_mean=False):
        qz_m, qz_v, z = self.encoder(x) # giving out mean, std and z because we want to use mean for the latent space
        latent = z if not use_z_mean else qz_m
#        p = self.decoder(latent)
        p = latent @ self.bp.T.float()
        p = self.Sigmoid(p + self.bias)
        return qz_m, qz_v, z, p

    def general_step(self, batch, batch_idx):
        x = batch.get("X_gex")
        y = batch.get("X")
        qz_m, qz_v, z, p = self.forward(x)
        bce = torch.nn.BCELoss(reduction="none")(p, y.float()).sum(dim=-1).sum()
        #qz_v is the variance with qz_v = sigma ^2
        #sigma = torch.sqrt(qz_v)
        kld = kl_divergence(Normal(qz_m, torch.sqrt(qz_v)), Normal(0, 1)).sum(dim=1)
        loss = bce.sum() + (kld ).sum()
        return loss, bce.sum(), kld.sum()
    
    def training_step(self, batch, batch_idx):
        loss, bce, kld = self.general_step(batch, batch_idx)
        self.log('loss', loss)
        self.log('BCE', bce)
        self.log('KLD', kld)
        return {'loss': loss}

    def validation_step(self, batch, batch_idx):
        loss, bce, kld = self.general_step(batch, batch_idx)
        self.log('val_loss', loss)
        self.log('val_BCE', bce)
        self.log('val_KLD', kld)
        return {'loss': loss}  

    def configure_optimizers(self):
        optim = torch.optim.Adam(
                    self.parameters(),
                    lr = hparams["lr"],
                    betas=(0.9, 0.999), eps=1e-08, weight_decay=hparams["wd"], amsgrad=False)
        return optim

In [ ]:
hparams = {"lr": 5e-4, #1e-3
           "wd": 1e-4,
#           "klw": 400,
           "n_layers": 3,
           "batch_size": 128,
           "latent_dim": 136,
           "n_hidden_encoder": 300, #116, #int(sqrt(gex.shape[1]))
#           "n_hidden_decoder": 342, #int(sqrt(adata.shape[1]))
           "binding_probabilities": binding_probs,
           }

In [ ]:
G2A = GexToATAC(hparams)

In [ ]:
scvi.data.setup_anndata(adata)
scvi.data.register_tensor_from_anndata(
            adata,
            registry_key='X_gex',
            adata_attr_name='obsm',
            adata_key_name='X_gex')

In [ ]:
data_splitter = DataSplitter(
    adata,
    train_size=0.9,
    validation_size=0.1,
    batch_size=hparams["batch_size"],
    use_gpu=True,
)

## Training

Use this comand in the ``CMD`` set to the proper directory: ``tensorboard --logdir lightning_logs --port 6005``

In [ ]:
trainer = pl.Trainer(
            max_epochs=25,
            gpus=1 if torch.cuda.is_available() else None
          )

G2A.to(device)
trainer.fit(G2A,
            data_splitter)

In [ ]:
#trainer.save_checkpoint("/mnt/CMSCB/CMSCB/GenEx to ATAC/our_save_dim3.ckpt")

In [ ]:
#G2A = GexToATAC.load_from_checkpoint("/mnt/CMSCB/CMSCB/GenEx to ATAC/our_save_dim3.ckpt", hparams = hparams)

## Evaluate Performance

## Load test data

In [ ]:
data = AnnDataLoader(adata, shuffle=False, batch_size=adata.shape[0])

In [ ]:
test_adata = test_adata[0:20000:2]
test_adata = test_adata.copy()

In [ ]:
scvi.data.setup_anndata(test_adata)
scvi.data.register_tensor_from_anndata(
            test_adata,
            registry_key='X_gex',
            adata_attr_name='obsm',
            adata_key_name='X_gex')
test_data = AnnDataLoader(test_adata, shuffle=False, batch_size=test_adata.shape[0]) 

In [ ]:
for batch in test_data:
    G2A.eval()
    with torch.no_grad():
        _,_,_,p  = G2A(batch.get("X_gex"))
        
test_adata.obsm["predicted_probs"] = p

## Precision and Recall

In [ ]:
precision,recall,_ = sklearn.metrics.precision_recall_curve(np.reshape(np.array(test_adata.X.todense()), -1),
                                                            np.reshape(np.array(test_adata.obsm["predicted_probs"]), -1))

In [ ]:
sklearn.metrics.PrecisionRecallDisplay(precision=precision, recall=recall).plot()

In [ ]:
AUPRC = sklearn.metrics.average_precision_score(np.reshape(np.array(test_adata.X.todense()), -1),
                                                np.reshape(np.array(test_adata.obsm["predicted_probs"]), -1))
AUPRC

In [ ]:
AUROC=sklearn.metrics.roc_auc_score(np.reshape(np.array(test_adata.X.todense()), -1),
                                    np.reshape(np.array(test_adata.obsm["predicted_probs"]), -1))
AUROC

In [ ]:
diff = np.array(test_adata.X.todense()) - np.array(test_adata.obsm["predicted_probs"])
n,m = test_adata.shape
RMSE = np.sqrt(1/(n * m) * (diff ** 2).sum())
RMSE

if our model would always predict 0 :

In [ ]:
np.sqrt(1/(n * m) * (np.array(test_adata.X.todense()) ** 2).sum())

Worst possibele RMSE = 1 if we always predinct the opposit.

## Latent Visualization

In [ ]:
for batch in test_data:
    G2A.eval()
    with torch.no_grad():
        _,_,latent  = G2A.encoder(batch.get("X_gex"))
test_adata.obsm["latent"] = latent

In [ ]:
def compute_embedding(adata, X_emb):
            
    adata.obsm['X_emb'] = X_emb
    
    if 'X_umap' in adata.obsm.keys():
        adata.obsm.pop('X_umap')
    
    if 'umap' in adata.obsm.keys():
        adata.obsm.pop('umap')
        
    if 'neighbors' in adata.uns.keys():
        adata.uns.pop('neighbors')

    sc.pp.neighbors(adata, use_rep='X_emb')
    sc.tl.umap(adata)

### Compare Latent Embedding to Cell Types

In [ ]:
# compute the k-nearest-neighbor graph that is used in both clustering and umap algorithms
sc.pp.neighbors(test_adata, use_rep="latent")

# compute the umap
sc.tl.umap(test_adata, min_dist=0.2)

# cluster the space (use a lower resolution to get fewer clusters than the default)
sc.tl.leiden(test_adata, key_added="our_cluster", resolution=1)
sc.pl.umap(test_adata, color='our_cluster')

In [ ]:
sklearn.metrics.adjusted_rand_score(test_adata.obs["our_cluster"], test_adata.obs["cell_type"])

# Regional Factor Correlation Plot

In [ ]:
import scipy
import matplotlib.pyplot as plt

In [ ]:
mean_access=np.mean(adata.X, axis=0)
test_mean_access=np.mean(test_adata.X, axis=0)

In [ ]:
plt.scatter(np.squeeze(np.array(-np.log((1/mean_access)-1))),
            np.squeeze(G2A.bias.detach().numpy()),
            s=20, alpha=0.4)

plt.title("Correlation Region Factor - Average Accessesability")
plt.xlabel("Average Accessability in Train Data")
plt.ylabel("Region Factor")

In [ ]:
plt.scatter(np.squeeze(np.array(-np.log((1/test_mean_access)-1))),
            np.squeeze(G2A.bias.detach().numpy()),
            s=20, alpha=0.4)

plt.title("Correlation Region Factor - Average Accessesability")
plt.xlabel("Average Accessability in Test Data")
plt.ylabel("Region Factor")

In [ ]:
scipy.stats.pearsonr(np.squeeze(np.array(-np.log((1/mean_access)-1))),
                     np.squeeze(G2A.bias.detach().numpy()))

In [ ]:
np.min(test_mean_access)

In [ ]:
eps = 1e-6

In [ ]:
scipy.stats.pearsonr(np.squeeze(np.array(-np.log((1/(test_mean_access + eps))-1))),
                     np.squeeze(G2A.bias.detach().numpy()))

We added a small offset to mean accessability on test since some peaks are not accessable in the test set leading to 0 average accessability for that peak and thus leading to devision by 0 in the calculation of correlation.

## Transcription Factor Activities

In [ ]:
TF_names

In [ ]:
for i in range(0, 136):
    string = TF_names[i] + "-activity"
    test_adata.obs[string[20:]] = test_adata.obsm["latent"][:,i]

In [ ]:
TF_names_array = [None]*136

for i in range(0, 136):
    string = TF_names[i] + "-activity"
    TF_names_array[i] = string[20:]

TF_names_array

In [ ]:
compute_embedding(test_adata, latent)
sc.pl.umap(test_adata, color='cell_type')

In [ ]:
sc.pl.umap(test_adata, color = ["SPI1-activity", "GATA1-activity"])

In [ ]:
sc.pl.umap(test_adata, color = ["PAX5-activity", "TAL1-activity"])